## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Nov. 6th Nov. 13th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,November-6-2024,Condo/Co-op,600 NE 2nd St #313,Dania Beach,FL,33004.0,150000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Dania/600-NE-2nd-St-...,MARMLS,A11673812,N,Y,26.055532,-80.134201


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
62,PAST SALE,2024-11-11,Condo/Co-op,1865 S Ocean Dr Unit 5A,Hallandale Beach,FL,33009.0,370000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/186...,MARMLS,A11595480,N,Y,25.983740,-80.120430
33,PAST SALE,2024-11-08,Condo/Co-op,6521 Bay Club Dr Unit 6521-3,Fort Lauderdale,FL,33308.0,260000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/6521...,Beaches MLS,F10447471,N,Y,26.207562,-80.104131
75,PAST SALE,2024-11-08,Condo/Co-op,6000 NE 22nd Way Unit 3F,Fort Lauderdale,FL,33308.0,161000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/6000...,Beaches MLS,F10430234,N,Y,26.203049,-80.110186
23,PAST SALE,2024-11-08,Condo/Co-op,1720 Jefferson St #508,Hollywood,FL,33020.0,180000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1720-Jeffe...,Beaches MLS,F10462030,N,Y,26.005751,-80.141656
48,PAST SALE,2024-11-08,Condo/Co-op,1817 S Ocean Dr #621,Hallandale Beach,FL,33009.0,340000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/181...,MARMLS,A11641110,N,Y,25.985349,-80.120881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,PAST SALE,2024-11-04,Condo/Co-op,2871 Somerset Dr #205,Lauderdale Lakes,FL,33311.0,90000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-Lakes/287...,Beaches MLS,F10444482,N,Y,26.158620,-80.191430
35,PAST SALE,2024-11-04,Condo/Co-op,12900 SW 13th St Unit 307E,Pembroke Pines,FL,33027.0,195000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Pines/12900...,MARMLS,A11637104,N,Y,25.996288,-80.318830
55,PAST SALE,2024-11-04,Condo/Co-op,7355 NW 4th Pl #206,Margate,FL,33063.0,174999.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/7355-NW-4th-...,Beaches MLS,F10448175,N,Y,26.236270,-80.223915
38,PAST SALE,2024-11-04,Condo/Co-op,3020 NE 32nd Ave #1421,Fort Lauderdale,FL,33308.0,395000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3020...,Beaches MLS,F10456831,N,Y,26.166000,-80.103194


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

88


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$340,709


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$292


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$29,982,399


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Broward County
Input Previous Week Condo Sales Total (number units sold):
149
Input Previous Week Condo Average Sales Price:
328285
Input Previous Week Condo Average PSF:
263
Input Previous Week Condo Sales total (ex: 198_000_000)
48_900_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-04,Condo/Co-op,212 Briny Ave Unit A4,Pompano Beach,FL,33062.0,1150000.0,2.0,2.0,212 Briny Ave Condo,1589.0,NaN,1979.0,NaN,724.0,606.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/212-Br...,Beaches MLS,RX-11010086,N,Y,26.229694,-80.089671,https://www.redfin.com/FL/Pompano-Beach/212-Br...,Wahkuna Vega-Farrell,Compass Florida LLC (PB),None,None,Kyshana Guzman,Keller Williams Realty SW,None,None,1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-04,Condo/Co-op,212 Briny Ave Unit A4,Pompano Beach,FL,33062.0,1150000.0,2.0,2.0,212 Briny Ave Condo,1589.0,NaN,1979.0,NaN,724.0,606.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/212-Br...,Beaches MLS,RX-11010086,N,Y,26.229694,-80.089671,https://www.redfin.com/FL/Pompano-Beach/212-Br...,Wahkuna Vega-Farrell,Compass Florida LLC (PB),None,None,Kyshana Guzman,Keller Williams Realty SW,None,None,1,orange
1,PAST SALE,2024-11-08,Condo/Co-op,2051 SE 3rd St #407,Deerfield Beach,FL,33441.0,1125000.0,3.0,2.5,Ocean Plaza,2056.0,NaN,2004.0,NaN,547.0,1457.0,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/2051...,Beaches MLS,RX-10969814,N,Y,26.312078,-80.076524,https://www.redfin.com/FL/Deerfield-Beach/2051...,Stephen Katz,Keller Williams Realty Boca Raton,Karen Anderson,Keller Williams Realty Boca Raton,Ximena Penuela,"Fort Realty, LLC",None,None,2,blue
2,PAST SALE,2024-11-05,Condo/Co-op,1850 S Ocean Dr #3910,Hallandale Beach,FL,33009.0,1100000.0,2.0,2.0,BEACH CLUB CONDO,1782.0,NaN,2005.0,NaN,617.0,1835.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11642790,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Alexandra Hazen,London Foster Realty,None,None,Tatsiana Hladkaya,London Foster Realty,None,None,3,blue
3,PAST SALE,2024-11-04,Condo/Co-op,1850 S Ocean Dr #1105,Hallandale Beach,FL,33009.0,1050000.0,2.0,2.0,BEACH CLUB CONDO,1782.0,NaN,2005.0,NaN,589.0,1832.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11643655,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Andre Lauzier,Coldwell Banker Realty,None,None,Ada Sasson,Cervera Real Estate Inc,None,None,4,blue
4,PAST SALE,2024-11-06,Condo/Co-op,2001 N Ocean Blvd #1505,Fort Lauderdale,FL,33305.0,950000.0,3.0,3.0,The Vue,1685.0,NaN,2005.0,NaN,564.0,2003.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2001...,Beaches MLS,F10443774,N,Y,26.153527,-80.100717,https://www.redfin.com/FL/Fort-Lauderdale/2001...,David Sullivan,LoKation,None,None,Carmen Mingolelli,Douglas Elliman,None,None,5,blue
5,PAST SALE,2024-11-04,Condo/Co-op,1201 S Ocean Dr Unit 2401N,Hollywood,FL,33019.0,821000.0,3.0,2.5,SUMMIT CONDO,1990.0,NaN,1982.0,NaN,413.0,1395.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,MARMLS,A11655855,N,Y,26.006228,-80.117346,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,Sandra Rathe,Keller Williams Legacy,Kathleen Gannon,Keller Williams Legacy,Sandra Rathe,Keller Williams Legacy,None,None,6,blue
6,PAST SALE,2024-11-08,Condo/Co-op,1530 S Ocean Blvd #203,Lauderdale By The Sea,FL,33062.0,750000.0,2.0,2.0,Ocean East,1250.0,NaN,1960.0,NaN,600.0,1352.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1530-S...,Beaches MLS,F10432066,N,Y,26.211281,-80.092526,https://www.redfin.com/FL/Pompano-Beach/1530-S...,Marie Abbott,Prestige Waterfront Realty,None,None,Mark Pancallo,Galt Ocean Realty Inc,None,None,7,blue
7,PAST SALE,2024-11-06,Condo/Co-op,2201 S Ocean Dr #2703,Hollywood,FL,33019.0,730000.0,2.0,2.0,QUADOMAIN TOWER I AND IV,1500.0,NaN,1974.0,NaN,487.0,1228.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2201-S-Oce...,MARMLS,A11666994,N,Y,25.999000,-80.117494,https://www.redfin.com/FL/Hollywood/2201-S-Oce...,Osvaldo Cesar,Optimar International Realty,None,None,Osvaldo Cesar,Optimar International Realty,None,None,8,blue
8,PAST SALE,2024-11-06,Condo/Co-op,2742 Kinsington Cir Unit 24-1,Weston,FL,33332.0,700000.0,3.0,2.0,LEGENDS AT WESTON HILLS,1560.0,NaN,2002.0,NaN,449.0,700.0,Sold,NaN,NaN,ht

In [70]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'Nov. 3rd — Nov. 10th')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [71]:
m.save('index.html')

## Data snagger

In [32]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [33]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-04,Condo/Co-op,212 Briny Ave Unit A4,Pompano Beach,FL,33062.0,1150000.0,2.0,2.0,212 Briny Ave Condo,1589.0,NaN,1979.0,NaN,724.0,606.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/212-Br...,Beaches MLS,RX-11010086,N,Y,26.229694,-80.089671,https://www.redfin.com/FL/Pompano-Beach/212-Br...,Wahkuna Vega-Farrell,Compass Florida LLC (PB),None,None,Kyshana Guzman,Keller Williams Realty SW,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [34]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_11112024


# CREATE TEMPLATE 

In [35]:
muni_set = set(df_top_ten['CITY'])

In [36]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [37]:
df_top_ten.reset_index(inplace=True,drop=True)

In [38]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [39]:
top_sale

'212 Briny Ave Condo at 212 Briny Ave Unit A4 in Pompano Beach'

In [40]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [41]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [42]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-04,Condo/Co-op,212 Briny Ave Unit A4,Pompano Beach,FL,33062.0,1150000.0,2.0,2.0,212 Briny Ave Condo,1589.0,NaN,1979.0,NaN,724.0,606.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/212-Br...,Beaches MLS,RX-11010086,N,Y,26.229694,-80.089671,https://www.redfin.com/FL/Pompano-Beach/212-Br...,Wahkuna Vega-Farrell,Compass Florida LLC (PB),None,None,Kyshana Guzman,Keller Williams Realty SW,None,None,1,orange,212 Briny Ave Condo at 212 Briny Ave Unit A4 i...
1,PAST SALE,2024-11-08,Condo/Co-op,2051 SE 3rd St #407,Deerfield Beach,FL,33441.0,1125000.0,3.0,2.5,Ocean Plaza,2056.0,NaN,2004.0,NaN,547.0,1457.0,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/2051...,Beaches MLS,RX-10969814,N,Y,26.312078,-80.076524,https://www.redfin.com/FL/Deerfield-Beach/2051...,Stephen Katz,Keller Williams Realty Boca Raton,Karen Anderson,Keller Williams Realty Boca Raton,Ximena Penuela,"Fort Realty, LLC",None,None,2,blue,Ocean Plaza at 2051 SE 3rd St #407 in Deerfiel...
2,PAST SALE,2024-11-05,Condo/Co-op,1850 S Ocean Dr #3910,Hallandale Beach,FL,33009.0,1100000.0,2.0,2.0,BEACH CLUB CONDO,1782.0,NaN,2005.0,NaN,617.0,1835.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11642790,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Alexandra Hazen,London Foster Realty,None,None,Tatsiana Hladkaya,London Foster Realty,None,None,3,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #3910 in H...
3,PAST SALE,2024-11-04,Condo/Co-op,1850 S Ocean Dr #1105,Hallandale Beach,FL,33009.0,1050000.0,2.0,2.0,BEACH CLUB CONDO,1782.0,NaN,2005.0,NaN,589.0,1832.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11643655,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Andre Lauzier,Coldwell Banker Realty,None,None,Ada Sasson,Cervera Real Estate Inc,None,None,4,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #1105 in H...
4,PAST SALE,2024-11-06,Condo/Co-op,2001 N Ocean Blvd #1505,Fort Lauderdale,FL,33305.0,950000.0,3.0,3.0,The Vue,1685.0,NaN,2005.0,NaN,564.0,2003.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2001...,Beaches MLS,F10443774,N,Y,26.153527,-80.100717,https://www.redfin.com/FL/Fort-Lauderdale/2001...,David Sullivan,LoKation,None,None,Carmen Mingolelli,Douglas Elliman,None,None,5,blue,The Vue at 2001 N Ocean Blvd #1505 in Fort Lau...
5,PAST SALE,2024-11-04,Condo/Co-op,1201 S Ocean Dr Unit 2401N,Hollywood,FL,33019.0,821000.0,3.0,2.5,SUMMIT CONDO,1990.0,NaN,1982.0,NaN,413.0,1395.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,MARMLS,A11655855,N,Y,26.006228,-80.117346,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,Sandra Rathe,Keller Williams Legacy,Kathleen Gannon,Keller Williams Legacy,Sandra Rathe,Keller Williams Legacy,None,None,6,blue,SUMMIT CONDO at 1201 S Ocean Dr Unit 2401N in ...
6,PAST SALE,2024-11-08,Condo/Co-op,1530 S Ocean Blvd #203,Lauderdale By The Sea,FL,33062.0,750000.0,2.0,2.0,Ocean East,1250.0,NaN,1960.0,NaN,600.0,1352.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1530-S...,Beaches MLS,F10432066,N,Y,26.211281,-80.092526,https://www.redfin.com/FL/Pompano-Beach/1530-S...,Marie Abbott,Prestige Waterfront Realty,None,None,Mark Pancallo,Galt Ocean Realty Inc,None,None,7,blue,Ocean East at 1530 S Ocean Blvd #203 in Lauder...
7,PAST SALE,2024-11-06,Condo/Co-op,2201 S Ocean Dr #2703,Hollywood,FL,33019.0,730000.0,2.0,2.0,QUADOMAIN TOWER I AND IV,1500.0,NaN,1974.0,NaN,487.0,1228.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2201-S-Oce...,MARMLS,A11666994,N

In [43]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: 212 Briny Ave Condo closing tops Broward County weekly condo sales 
DEK: Top 10 sales ranged from $650,000M to $1,150,000M
FEATURED HED:
SEO HED: Broward County County Weekly Condo Report 
SEO DESCRIPTION: Broward County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward County, weekly condo sales, Weston, Fort Lauderdale, Lauderdale By The Sea, Hallandale Beach, Hollywood, Deerfield Beach, Pompano Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County County’s NEWS PEG HERE.

Brokers closed 88 condo sales totaling $29,982,399 million from Nov. 6th to Nov. 13th. The previous week, brokers closed 149 condo sales totaling $48,900,000.

Last week’s units sold for an average of $340,709, higher th

In [44]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [45]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [46]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [47]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-04,Condo/Co-op,212 Briny Ave Unit A4,Pompano Beach,FL,33062.0,1150000.0,2.0,2.0,212 Briny Ave Condo,1589.0,NaN,1979.0,NaN,724.0,606.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/212-Br...,Beaches MLS,RX-11010086,N,Y,26.229694,-80.089671,https://www.redfin.com/FL/Pompano-Beach/212-Br...,Wahkuna Vega-Farrell,Compass Florida LLC (PB),None,None,Kyshana Guzman,Keller Williams Realty SW,None,None,1,orange,212 Briny Ave Condo at 212 Briny Ave Unit A4 i...
1,PAST SALE,2024-11-08,Condo/Co-op,2051 SE 3rd St #407,Deerfield Beach,FL,33441.0,1125000.0,3.0,2.5,Ocean Plaza,2056.0,NaN,2004.0,NaN,547.0,1457.0,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/2051...,Beaches MLS,RX-10969814,N,Y,26.312078,-80.076524,https://www.redfin.com/FL/Deerfield-Beach/2051...,Stephen Katz,Keller Williams Realty Boca Raton,Karen Anderson,Keller Williams Realty Boca Raton,Ximena Penuela,"Fort Realty, LLC",None,None,2,blue,Ocean Plaza at 2051 SE 3rd St #407 in Deerfiel...
2,PAST SALE,2024-11-05,Condo/Co-op,1850 S Ocean Dr #3910,Hallandale Beach,FL,33009.0,1100000.0,2.0,2.0,BEACH CLUB CONDO,1782.0,NaN,2005.0,NaN,617.0,1835.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11642790,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Alexandra Hazen,London Foster Realty,None,None,Tatsiana Hladkaya,London Foster Realty,None,None,3,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #3910 in H...
3,PAST SALE,2024-11-04,Condo/Co-op,1850 S Ocean Dr #1105,Hallandale Beach,FL,33009.0,1050000.0,2.0,2.0,BEACH CLUB CONDO,1782.0,NaN,2005.0,NaN,589.0,1832.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11643655,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Andre Lauzier,Coldwell Banker Realty,None,None,Ada Sasson,Cervera Real Estate Inc,None,None,4,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #1105 in H...
4,PAST SALE,2024-11-06,Condo/Co-op,2001 N Ocean Blvd #1505,Fort Lauderdale,FL,33305.0,950000.0,3.0,3.0,The Vue,1685.0,NaN,2005.0,NaN,564.0,2003.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2001...,Beaches MLS,F10443774,N,Y,26.153527,-80.100717,https://www.redfin.com/FL/Fort-Lauderdale/2001...,David Sullivan,LoKation,None,None,Carmen Mingolelli,Douglas Elliman,None,None,5,blue,The Vue at 2001 N Ocean Blvd #1505 in Fort Lau...
5,PAST SALE,2024-11-04,Condo/Co-op,1201 S Ocean Dr Unit 2401N,Hollywood,FL,33019.0,821000.0,3.0,2.5,SUMMIT CONDO,1990.0,NaN,1982.0,NaN,413.0,1395.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,MARMLS,A11655855,N,Y,26.006228,-80.117346,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,Sandra Rathe,Keller Williams Legacy,Kathleen Gannon,Keller Williams Legacy,Sandra Rathe,Keller Williams Legacy,None,None,6,blue,SUMMIT CONDO at 1201 S Ocean Dr Unit 2401N in ...
6,PAST SALE,2024-11-08,Condo/Co-op,1530 S Ocean Blvd #203,Lauderdale By The Sea,FL,33062.0,750000.0,2.0,2.0,Ocean East,1250.0,NaN,1960.0,NaN,600.0,1352.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1530-S...,Beaches MLS,F10432066,N,Y,26.211281,-80.092526,https://www.redfin.com/FL/Pompano-Beach/1530-S...,Marie Abbott,Prestige Waterfront Realty,None,None,Mark Pancallo,Galt Ocean Realty Inc,None,None,7,blue,Ocean East at 1530 S Ocean Blvd #203 in Lauder...
7,PAST SALE,2024-11-06,Condo/Co-op,2201 S Ocean Dr #2703,Hollywood,FL,33019.0,730000.0,2.0,2.0,QUADOMAIN TOWER I AND IV,1500.0,NaN,1974.0,NaN,487.0,1228.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2201-S-Oce...,MARMLS,A11666994,N

In [48]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: 212 Briny Ave Condo closing tops Broward County weekly condo sales 
DEK: Top 10 sales ranged from $650,000M to $1,150,000M
FEATURED HED:
SEO HED: Broward County County Weekly Condo Report 
SEO DESCRIPTION: Broward County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward County, weekly condo sales, Weston, Fort Lauderdale, Lauderdale By The Sea, Hallandale Beach, Hollywood, Deerfield Beach, Pompano Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County County’s NEWS PEG HERE.

Brokers closed 88 condo sales totaling $29,982,399 million from Nov. 6th to Nov. 13th. The previous week, brokers closed 149 condo sales totaling $48,900,000.

Last week’s units sold for an average of $340,709, higher th

In [56]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Pompano-Beach/212-Briny-Ave-33062/unit-A4/home/41701545


In [72]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Deerfield-Beach/2051-SE-3rd-St-33441/unit-407/home/42010129


In [73]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Fort-Lauderdale/3900-Galt-Ocean-Dr-33308/unit-1803/home/41709362


In [74]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Weston/2742-Kinsington-Cir-33332/unit-24-1/home/42004276


In [75]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Pompano-Beach/212-Briny-Ave-33062/unit-A4/home/41701545


In [76]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Hollywood/1201-S-Ocean-Dr-33019/unit-2401N/home/41934710


In [77]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-04,Condo/Co-op,212 Briny Ave Unit A4,Pompano Beach,FL,33062.0,1150000.0,2.0,2.0,212 Briny Ave Condo,1589.0,NaN,1979.0,NaN,724.0,606.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/212-Br...,Beaches MLS,RX-11010086,N,Y,26.229694,-80.089671,https://www.redfin.com/FL/Pompano-Beach/212-Br...,Wahkuna Vega-Farrell,Compass Florida LLC (PB),None,None,Kyshana Guzman,Keller Williams Realty SW,None,None,1,orange,212 Briny Ave Condo at 212 Briny Ave Unit A4 i...
1,PAST SALE,2024-11-08,Condo/Co-op,2051 SE 3rd St #407,Deerfield Beach,FL,33441.0,1125000.0,3.0,2.5,Ocean Plaza,2056.0,NaN,2004.0,NaN,547.0,1457.0,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/2051...,Beaches MLS,RX-10969814,N,Y,26.312078,-80.076524,https://www.redfin.com/FL/Deerfield-Beach/2051...,Stephen Katz,Keller Williams Realty Boca Raton,Karen Anderson,Keller Williams Realty Boca Raton,Ximena Penuela,"Fort Realty, LLC",None,None,2,blue,Ocean Plaza at 2051 SE 3rd St #407 in Deerfiel...
2,PAST SALE,2024-11-05,Condo/Co-op,1850 S Ocean Dr #3910,Hallandale Beach,FL,33009.0,1100000.0,2.0,2.0,BEACH CLUB CONDO,1782.0,NaN,2005.0,NaN,617.0,1835.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11642790,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Alexandra Hazen,London Foster Realty,None,None,Tatsiana Hladkaya,London Foster Realty,None,None,3,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #3910 in H...
3,PAST SALE,2024-11-04,Condo/Co-op,1850 S Ocean Dr #1105,Hallandale Beach,FL,33009.0,1050000.0,2.0,2.0,BEACH CLUB CONDO,1782.0,NaN,2005.0,NaN,589.0,1832.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11643655,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Andre Lauzier,Coldwell Banker Realty,None,None,Ada Sasson,Cervera Real Estate Inc,None,None,4,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #1105 in H...
4,PAST SALE,2024-11-06,Condo/Co-op,2001 N Ocean Blvd #1505,Fort Lauderdale,FL,33305.0,950000.0,3.0,3.0,The Vue,1685.0,NaN,2005.0,NaN,564.0,2003.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2001...,Beaches MLS,F10443774,N,Y,26.153527,-80.100717,https://www.redfin.com/FL/Fort-Lauderdale/2001...,David Sullivan,LoKation,None,None,Carmen Mingolelli,Douglas Elliman,None,None,5,blue,The Vue at 2001 N Ocean Blvd #1505 in Fort Lau...
5,PAST SALE,2024-11-04,Condo/Co-op,1201 S Ocean Dr Unit 2401N,Hollywood,FL,33019.0,821000.0,3.0,2.5,SUMMIT CONDO,1990.0,NaN,1982.0,NaN,413.0,1395.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,MARMLS,A11655855,N,Y,26.006228,-80.117346,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,Sandra Rathe,Keller Williams Legacy,Kathleen Gannon,Keller Williams Legacy,Sandra Rathe,Keller Williams Legacy,None,None,6,blue,SUMMIT CONDO at 1201 S Ocean Dr Unit 2401N in ...
6,PAST SALE,2024-11-08,Condo/Co-op,1530 S Ocean Blvd #203,Lauderdale By The Sea,FL,33062.0,750000.0,2.0,2.0,Ocean East,1250.0,NaN,1960.0,NaN,600.0,1352.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1530-S...,Beaches MLS,F10432066,N,Y,26.211281,-80.092526,https://www.redfin.com/FL/Pompano-Beach/1530-S...,Marie Abbott,Prestige Waterfront Realty,None,None,Mark Pancallo,Galt Ocean Realty Inc,None,None,7,blue,Ocean East at 1530 S Ocean Blvd #203 in Lauder...
7,PAST SALE,2024-11-06,Condo/Co-op,2201 S Ocean Dr #2703,Hollywood,FL,33019.0,730000.0,2.0,2.0,QUADOMAIN TOWER I AND IV,1500.0,NaN,1974.0,NaN,487.0,1228.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2201-S-Oce...,MARMLS,A11666994,N

## Time on Market Calculator

In [69]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 8, 1) ## List (Earlier) date
date2 = datetime(2024, 11, 4) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

95


## Clean CSV for Datawrapper Chart

In [78]:
chart_df = df_top_ten

In [79]:
chart_df = chart_df.fillna(" ")

In [80]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [81]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [82]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [83]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [84]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [85]:
csv_date_string = today.strftime("%m_%d_%Y")

In [86]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [87]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-11-04,212 Briny Ave Unit A4 in Pompano Beach,"$1,150,000",2,2.0,"1,589",1979,$724,Beaches MLS,RX-11010086,Wahkuna Vega-Farrell Compass Florida LLC (PB),Kyshana Guzman Keller Williams Realty SW
1,2024-11-08,2051 SE 3rd St #407 in Deerfield Beach,"$1,125,000",3,2.5,"2,056",2004,$547,Beaches MLS,RX-10969814,Stephen Katz Keller Williams Realty Boca Raton...,"Ximena Penuela Fort Realty, LLC"
2,2024-11-05,1850 S Ocean Dr #3910 in Hallandale Beach,"$1,100,000",2,2.0,"1,782",2005,$617,MARMLS,A11642790,Alexandra Hazen London Foster Realty,Tatsiana Hladkaya London Foster Realty
3,2024-11-04,1850 S Ocean Dr #1105 in Hallandale Beach,"$1,050,000",2,2.0,"1,782",2005,$589,MARMLS,A11643655,Andre Lauzier Coldwell Banker Realty,Ada Sasson Cervera Real Estate Inc
4,2024-11-06,2001 N Ocean Blvd #1505 in Fort Lauderdale,"$950,000",3,3.0,"1,685",2005,$564,Beaches MLS,F10443774,David Sullivan LoKation,Carmen Mingolelli Douglas Elliman
5,2024-11-04,1201 S Ocean Dr Unit 2401N in Hollywood,"$821,000",3,2.5,"1,990",1982,$413,MARMLS,A11655855,Sandra Rathe Keller Williams Legacy Kathleen G...,Sandra Rathe Keller Williams Legacy
6,2024-11-08,1530 S Ocean Blvd #203 in Lauderdale By The Sea,"$750,000",2,2.0,"1,250",1960,$600,Beaches MLS,F10432066,Marie Abbott Prestige Waterfront Realty,Mark Pancallo Galt Ocean Realty Inc
7,2024-11-06,2201 S Ocean Dr #2703 in Hollywood,"$730,000",2,2.0,"1,500",1974,$487,MARMLS,A11666994,Osvaldo Cesar Optimar International Realty,Osvaldo Cesar Optimar International Realty
8,2024-11-06,2742 Kinsington Cir Unit 24-1 in Weston,"$700,000",3,2.0,"1,560",2002,$449,MARMLS,A11654867,Maria Diodato Brozzi Avanti Way Brickell,Maria Diodato Brozzi Avanti Way Brickell
9,2024-11-06,3900 Galt Ocean Dr #1803 in Fort Lauderdale,"$650,000",2,2.0,"1,435",1975,$453,Beaches MLS,F10415052,John Munroe Munroe Coastal Propeties,Roderick Perkins Luxury Real Estate Group LLC
